# Convert image of groceries into semi-structured text and insert into delta table

## Configuration and Setup

In [1]:
# Import libaries
import os
import pandas as pd
from pyspark.sql import SparkSession
from synapse.ml.core.platform import running_on_synapse, find_secret
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
import json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import re
import datetime
from openai import AzureOpenAI

StatementMeta(, 270fe012-7360-40cb-800f-7799068f1796, 5, Finished, Available)

In [2]:
#Azure AI Search
AI_SEARCH_ENDPOINT = "https://fabrichackathonsearch.search.windows.net"
AI_SEARCH_INDEX_NAME = "hackindex3"
#AI_SEARCH_API_KEY = <keyvault>
#Azure AI Services
AI_SERVICE_ENDPOINT = "https://fabrichackathonopenai2.openai.azure.com/"
#AI_SERVICE_KEY = <keyvault>
AI_SERVICE_LOCATION = "swedencentral"
AI_SERVICE_VISION_DEPLOYMENT = "gpt4vision"
AI_SERVICE_DEPLOYMENT = "gpt4"
AI_SERVICE_EMBEDDING_DEPLOYMENT = "embeddingada"
AI_SERVICE_API = "2023-12-01-preview"

StatementMeta(, 270fe012-7360-40cb-800f-7799068f1796, 6, Finished, Available)

## Define parameters to hand-over

In [3]:
image_file_name = 'einkaufswagen.jpg'

StatementMeta(, 270fe012-7360-40cb-800f-7799068f1796, 7, Finished, Available)

In [4]:
image_url = f'https://dlsfabrichackathon002.blob.core.windows.net/images/{image_file_name}'
print(image_url)

StatementMeta(, 270fe012-7360-40cb-800f-7799068f1796, 8, Finished, Available)

https://dlsfabrichackathon002.blob.core.windows.net/images/einkaufswagen.jpg


## Data Extraction with GPT-4-Vision-Preview

In [5]:
# Extract groceries from image

client = AzureOpenAI(
    api_key=AI_SERVICE_KEY,  
    api_version=AI_SERVICE_API,
    base_url=f"{AI_SERVICE_ENDPOINT}openai/deployments/{AI_SERVICE_VISION_DEPLOYMENT}/extensions",
)

response = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": 
          f"""
          What groceries can you see in the picture? 
          Answer in string form which can be used to create a JSON with the following information for each kind of grocery you can see: 
          product_name, brand_name, unit_name, metric_value. 
          For unit_name only use the following categories: ["liter", "kilogram", "gram", "milliliter", "centiliter"].
          You must at least estimate a metric value. It must be always a decimal value. 
          Do not add any extra information before or after the text which can be transformed in a JSON file.
          Make sure that the answer is a "list" which starts with "[{{" and ends with "}}]", also if the list has only one element. 
          """},
        {
          "type": "image_url",
          "image_url": {
            "url": image_url,
          },
        },
      ],
    }
  ],
  max_tokens=800,
)

image_content_str = response.choices[0].message.content


StatementMeta(, 270fe012-7360-40cb-800f-7799068f1796, 9, Finished, Available)

In [6]:
pattern = r'\[(.*)\]'
match2 = re.search(pattern, image_content_str, re.DOTALL)

if match2:
    # Extract the content within the brackets
    image_content_str = match2.group(0)
else:
    print("No match found")

StatementMeta(, 270fe012-7360-40cb-800f-7799068f1796, 10, Finished, Available)

In [7]:
print(image_content_str)

StatementMeta(, 270fe012-7360-40cb-800f-7799068f1796, 11, Finished, Available)

[{"product_name":"Tofu","brand_name":"BioBio","unit_name":"gram","metric_value":250.0},
{"product_name":"Vegan Muhlen Filet","brand_name":"Rügenwalder Mühle","unit_name":"gram","metric_value":180.0},
{"product_name":"Falafel Balls","brand_name":"Veganz","unit_name":"gram","metric_value":250.0},
{"product_name":"Peace Burger","brand_name":"Vossko","unit_name":"gram","metric_value":239.0},
{"product_name":"Aubergine Spread","brand_name":"BioBio","unit_name":"gram","metric_value":180.0},
{"product_name":"Nature One Yogurt","brand_name":"Alpro","unit_name":"gram","metric_value":500.0},
{"product_name":"Blueberry Yogurt","brand_name":"Alpro","unit_name":"gram","metric_value":500.0},
{"product_name":"Lentil Veggie Bolognese","brand_name":"Frosta","unit_name":"gram","metric_value":500.0},
{"product_name":"Shower Gel","brand_name":"Blutezeit","unit_name":"milliliter","metric_value":300.0},
{"product_name":"Tempranillo Wine","brand_name":"Tempranillo","unit_name":"liter","metric_value":0.75},
{

In [8]:
image_content_list = json.loads(image_content_str)
names = [item["product_name"] for item in image_content_list]

StatementMeta(, 270fe012-7360-40cb-800f-7799068f1796, 12, Finished, Available)

## RAG Enrichment of Data

In [9]:
# Create a client
credential = AzureKeyCredential(AI_SEARCH_API_KEY)
client = SearchClient(endpoint=AI_SEARCH_ENDPOINT,
                    index_name=AI_SEARCH_INDEX_NAME,
                    credential=credential)

promptstring = ""

for product in names:
    results = client.search(search_text=product, top=2)

    for result in results:
        result_str = str(result)
        #print(result)
        promptstring = promptstring + "/n" + result_str



StatementMeta(, 270fe012-7360-40cb-800f-7799068f1796, 13, Finished, Available)

In [10]:
print(promptstring)

StatementMeta(, 270fe012-7360-40cb-800f-7799068f1796, 14, Finished, Available)

/n{'KgCO2PerKg': '0.9', 'id': '46', 'ProductName': 'Firm Tofu', 'PackagingType': 'kg', 'Value': '1', '@search.score': 2.0745916, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}/n{'KgCO2PerKg': '1', 'id': '49', 'ProductName': 'Smoked Tofu', 'PackagingType': 'kg', 'Value': '1', '@search.score': 1.8249029, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}/n{'KgCO2PerKg': '1.3', 'id': '54', 'ProductName': 'Vegan Chicken Breast Substitute', 'PackagingType': 'kg', 'Value': '1', '@search.score': 1.685442, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}/n{'KgCO2PerKg': '1.5', 'id': '55', 'ProductName': 'Vegan Burger Patty', 'PackagingType': 'kg', 'Value': '1', '@search.score': 1.5052558, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}/n{'KgCO2PerKg': '1.5', 'id': '55', 'ProductName': 'Vegan Burger Patty', 'PackagingType': 'kg', 'Value': '1', '@se

In [11]:
client = AzureOpenAI(
    api_key=AI_SERVICE_KEY,  
    api_version=AI_SERVICE_API,
    base_url=f"{AI_SERVICE_ENDPOINT}openai/deployments/{AI_SERVICE_DEPLOYMENT}",
)

response = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": 
          f"""
          You are helping with setting up a power bi dashboard for co2 consumption. 
          In an earlier prompt you were analyzing a picture and giving back a json with the groceries you saw on the picture.
          This was your response: 

          {image_content_str}

          We searched for the groceries in the picture in a co2 knowledge base. This was the result of the query for all products:

          {promptstring}

          Return a json with the following information from the first response (use the exact same values) and the additional co2 column:
          kg_co2_per_metric_value. It must be always a decimal value. 
          E.g. if there was 2 liters of milk in the picture you calculate 2 x KgCO2PerKg for kg_co2_per_metric_value.
          If there's no information about the co2 metric than just add "none" as a value for kg_co2_per_metric_value. 
          Do not add any extra information before or after the text which can be transformed in a JSON file.SearchClient.
          Make sure that the answer is a "list" which starts with "[" and ends with "]", also if the list has only one element.
          Make sure the values included are exatly: "product_name", "brand_name", "unit_name", "metric_value", "kg_co2_per_metric_value"" 
          """},
      ],
    }
  ],
  max_tokens=1000,
)

image_content_enriched_str = response.choices[0].message.content

StatementMeta(, 270fe012-7360-40cb-800f-7799068f1796, 15, Finished, Available)

In [12]:
match3 = re.search(pattern, image_content_enriched_str, re.DOTALL)

if match3:
    # Extract the content within the brackets
    image_content_enriched_str = match3.group(0)
else:
    print("No match found")

StatementMeta(, 270fe012-7360-40cb-800f-7799068f1796, 16, Finished, Available)

In [13]:
print(image_content_enriched_str)

StatementMeta(, 270fe012-7360-40cb-800f-7799068f1796, 17, Finished, Available)

[{"product_name":"Tofu","brand_name":"BioBio","unit_name":"gram","metric_value":250,"kg_co2_per_metric_value":0.225},
{"product_name":"Vegan Muhlen Filet","brand_name":"Rügenwalder Mühle","unit_name":"gram","metric_value":180,"kg_co2_per_metric_value":0.234},
{"product_name":"Falafel Balls","brand_name":"Veganz","unit_name":"gram","metric_value":250,"kg_co2_per_metric_value":"none"},
{"product_name":"Peace Burger","brand_name":"Vossko","unit_name":"gram","metric_value":239,"kg_co2_per_metric_value":0.358},
{"product_name":"Aubergine Spread","brand_name":"BioBio","unit_name":"gram","metric_value":180,"kg_co2_per_metric_value":"none"},
{"product_name":"Nature One Yogurt","brand_name":"Alpro","unit_name":"gram","metric_value":500,"kg_co2_per_metric_value":"none"},
{"product_name":"Blueberry Yogurt","brand_name":"Alpro","unit_name":"gram","metric_value":500,"kg_co2_per_metric_value":"none"},
{"product_name":"Lentil Veggie Bolognese","brand_name":"Frosta","unit_name":"gram","metric_value":5

## Upsert data into silver lakehouse

In [14]:
image_content_enriched_list = json.loads(image_content_enriched_str)

StatementMeta(, 270fe012-7360-40cb-800f-7799068f1796, 18, Finished, Available)

In [15]:
#Create Pandas DataFrame
df_pd = pd.DataFrame(image_content_enriched_list)
df_pd.columns = ["product_name","brand_name","unit_name","metric_value", "kg_co2_per_metric_value"]
current_utc_time = datetime.datetime.utcnow()
df_pd['timestamp'] = current_utc_time
df_pd['image_url'] = image_url
df_pd = df_pd.astype(str)

df_sp = spark.createDataFrame(df_pd)
display(df_sp)

#Insert Data into Lakehouse
df_sp.write \
    .format("delta") \
    .mode("append") \
    .option("mergeSchema", "true") \
    .saveAsTable("enriched_groceries", path="Tables/enriched_groceries", mode="append")

StatementMeta(, 270fe012-7360-40cb-800f-7799068f1796, 19, Finished, Available)

SynapseWidget(Synapse.DataFrame, 7bc8fb63-cb10-4c24-bda1-e4cc9161887d)